In [1]:
!pip install "pymilvus[model]==2.5.10" openai==1.82.0 requests==2.32.3 tqdm==4.67.1 torch==2.7.0

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/


In [2]:
def get_default_embedding_model():
    """
    获取默认的嵌入模型
    """
    from pymilvus import model as milvus_model
    embedding_model = milvus_model.DefaultEmbeddingFunction()
    test_embedding = embedding_model.encode_queries(["test"])
    embedding_dim = len(test_embedding[0])

    from pymilvus import MilvusClient
    milvus_client = MilvusClient(uri="./mfd.db")

    return embedding_model, embedding_dim, milvus_client


embedding_model, embedding_dim, milvus_client = get_default_embedding_model()

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.11/site-packages/milvus_lite/__init__.py:15: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution


In [3]:
collection_name = "mfd_collection"


def create_milvus_collection():

    if milvus_client.has_collection(collection_name):
        milvus_client.drop_collection(collection_name)

    milvus_client.create_collection(
        collection_name=collection_name,
        dimension=embedding_dim,
        metric_type="IP",  # 内积距离
        consistency_level="Strong",  # 一致性相关
    )
    print(f"Created milvus collection: {collection_name}")


create_milvus_collection()

Created milvus collection: mfd_collection


In [4]:
mfd_file_path = "mfd.md"


def insert_data_from_files():
    text_lines = []
    file_paths = [mfd_file_path]
    for file_path in file_paths:
        with open(file_path, "r") as file:
            file_text = file.read()
        text_lines += file_text.split("# ")

    data = []
    doc_embeddings = embedding_model.encode_documents(text_lines)

    for i, line in enumerate(text_lines):
        data.append({"id": i, "vector": doc_embeddings[i], "text": line})
        print(f"creating embeddings: {i}, {line[:min(len(line),20)]}...")

    milvus_client.insert(collection_name=collection_name, data=data)
    print("mfd.md is inserted to db")


insert_data_from_files()

creating embeddings: 0, #...
creating embeddings: 1, 中华人民共和国民法典

##...
creating embeddings: 2, （二）物权编

###...
creating embeddings: 3, 第一章 一般规定

**第二百零四条**...
creating embeddings: 4, 第二章 所有权

####...
creating embeddings: 5, 第一节 一般规定

**第二百四十四条*...
creating embeddings: 6, 第二节 共有

**第三百一十条** 两...
creating embeddings: 7, 第三节 相邻关系

**第三百二十六条*...
creating embeddings: 8, 第四节 所有权取得的特别规定

**第三...
creating embeddings: 9, 第三章 用益物权

####...
creating embeddings: 10, 第一节 一般规定

**第三百五十三条*...
creating embeddings: 11, 第二节 土地承包经营权

**第三百五十...
creating embeddings: 12, 第三节 建设用地使用权

**第三百六十...
creating embeddings: 13, 第四节 宅基地使用权

**第三百七十七...
creating embeddings: 14, 第五节 居住权

**第三百八十二条**...
creating embeddings: 15, 第六节 地役权

**第三百八十八条**...
creating embeddings: 16, 第四章 担保物权

####...
creating embeddings: 17, 第一节 一般规定

**第三百九十九条*...
creating embeddings: 18, 第二节 抵押权

**第四百零九条** ...
creating embeddings: 19, 第三节 质权

#####...
creating embeddings: 20, 一、动产质权

**第四百三十四条** ...
creating embeddings: 21, 二、权利质权

**第四百四十九

In [5]:
def search_milvus_db(question):
    search_res = milvus_client.search(
        collection_name=collection_name,
        data=embedding_model.encode_queries(
            [question]
        ),  # 将问题转换为嵌入向量
        limit=5,  # 返回前5个结果
        search_params={
            "metric_type": "IP",   # 余弦相似度
            "params": {"nprobe": 16}   # 搜索的聚类中心数量，增大可提高召回率但降低性能
        },
        output_fields=["text"],  # 返回 text 字段
    )
    retrievals_with_distance = [
        (res["entity"]["text"], res["distance"]) for res in search_res[0]
    ]
    import json
    print("\n" + "=" * 20 + "RAG检索结果" + "=" * 20 + "\n")
    print(json.dumps(retrievals_with_distance, indent=4, ensure_ascii=False))
    context = "\n".join(
        [line_with_distance[0] for line_with_distance in retrievals_with_distance]
    )
    return context


In [8]:
def ask_llm_with_rag(question):
    context = search_milvus_db(question)
    SYSTEM_PROMPT = """
    Human: 你是一个 AI 助手。你能够从提供的上下文段落片段中找到问题的答案。
    """
    user_msg = f"""
    请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。
    <context>
    {context}
    </context>
    <question>
    {question}
    </question>
    """

    reasoning_content = ""  # 定义完整思考过程
    answer_content = ""     # 定义完整回复
    is_answering = False   # 判断是否结束思考过程并开始回复

    # 从环境变量获取 Dashscope API Key
    import os
    api_key = os.getenv("DASHSCOPE_API_KEY")
    if not api_key:
        raise ValueError("请设置 DASHSCOPE_API_KEY 环境变量")

    # 初始化 OpenAI 客户端
    from openai import OpenAI
    client = OpenAI(
        api_key=api_key,
        base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
    )

    # 创建聊天完成请求
    completion = client.chat.completions.create(
        model="deepseek-r1",  # 此处以 deepseek-r1 为例，可按需更换模型名称
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": user_msg}
        ],
        stream=True,
        # 解除以下注释会在最后一个chunk返回Token使用量
        # stream_options={
        #     "include_usage": True
        # }
    )

    print("\n" + "=" * 20 + "思考过程" + "=" * 20 + "\n")

    for chunk in completion:
        # 如果chunk.choices为空，则打印usage
        if not chunk.choices:
            print("\nUsage:")
            print(chunk.usage)
        else:
            delta = chunk.choices[0].delta
            # 打印思考过程
            if hasattr(delta, 'reasoning_content') and delta.reasoning_content != None:
                print(delta.reasoning_content, end='', flush=True)
                reasoning_content += delta.reasoning_content
            else:
                # 开始回复
                if delta.content != "" and is_answering == False:
                    print("\n" + "=" * 20 + "完整回复" + "=" * 20 + "\n")
                    is_answering = True
                # 打印回复过程
                print(delta.content, end='', flush=True)
                answer_content += delta.content
    return answer_content

In [9]:
ask_llm_with_rag("邻居装修导致承重墙开裂，《民法典》相邻关系条款中哪些规定可以适用？受害人可主张哪些救济？")


====================RAG检索结果====================

[
    [
        "二、权利质权\n\n**第四百四十九条** 可以出质的权利包括：\n（一）汇票、本票、支票；\n（二）债券、存款单；\n（三）仓单、提单；\n（四）可以转让的基金份额、股权；\n（五）可以转让的注册商标专用权、专利权、著作权等知识产权中的财产权；\n（六）应收账款；\n（七）法律、行政法规规定可以出质的其他财产权利。\n\n**第四百五十条** 以汇票、本票、支票、债券、存款单、仓单、提单出质的，当事人应当订立书面合同。质权自权利凭证交付之日起设立。\n\n**第四百五十一条** 以记名股票出质的，当事人应当订立书面合同。质权自股票交付之日起设立。\n以未上市公司股权出质的，适用公司法有关股权转让的规定。\n\n**第四百五十二条** 以可以转让的基金份额、股权出质的，当事人应当订立书面合同。质权自基金份额、股权登记于证券登记结算机构或者公司章程载明的股权登记簿时设立。\n以未上市公司股权出质的，适用公司法有关股权转让的规定。\n\n**第四百五十三条** 以可以转让的注册商标专用权、专利权、著作权等知识产权中的财产权出质的，当事人应当订立书面合同。质权自权利质押登记于相关部门时设立。\n\n**第四百五十四条** 以应收账款出质的，当事人应当订立书面合同。质权自应收账款质押登记于中国人民银行征信中心时设立。\n\n**第四百五十五条** 以法律、行政法规规定可以出质的其他财产权利出质的，依照法律、行政法规的规定。\n\n**第四百五十六条** 权利质权除适用本节规定外，参照适用本章动产质权的有关规定。\n\n####",
        0.6015046834945679
    ],
    [
        "第三章 合同的变更和转让\n\n**第五百四十八条** 当事人协商一致，可以变更合同。\n\n**第五百四十九条** 当事人对合同变更的内容约定不明确的，推定为未变更。\n\n**第五百五十条** 债权人可以将合同的权利全部或者部分转让给第三人，但是有下列情形之一的除外：\n（一）根据合同性质不得转让；\n（二）按照当事人约定不得转让；\n（三）依照法律规定不得转让。\n债权人转让权利的，应当通知债务人。未经通知，该转

'根据提供的上下文信息，**未包含《民法典》相邻关系条款的具体内容**（相邻关系条款通常规定在物权编的第七章）。当前上下文主要涉及权利质权、合同变更转让、居住权、担保物权和土地承包经营权等章节，**未直接包含相邻关系的法律规定**。\n\n因此，**无法直接从提供的条款中找出适用于“邻居装修导致承重墙开裂”的相邻关系规定及具体救济措施**。\n\n---\n\n### 建议参考《民法典》相邻关系核心条款（未在提供上下文中出现）：\n虽然提供的片段未涵盖，但《民法典》相邻关系核心条款通常包括：\n1. **第二百八十八条**：处理相邻关系的基本原则（有利生产、方便生活、团结互助、公平合理）。\n2. **第二百九十五条**：不动产权利人不得违反国家规定弃置固体废物，排放大气污染物、水污染物、土壤污染物、噪声、光辐射、电磁辐射等有害物质。\n3. **第二百九十六条**：不动产权利人因用水、排水、通行、铺设管线等利用相邻不动产的，应当尽量避免对相邻的不动产权利人造成损害。\n4. **第二百九十七条**：建造建筑物不得违反国家有关工程建设标准，不得妨碍相邻建筑物的通风、采光和日照。\n5. **第二百九十八条**：不动产权利人挖掘土地、建造建筑物、铺设管线以及安装设备等，不得危及相邻不动产的安全。\n\n---\n\n### 针对“承重墙开裂”的可主张救济（基于《民法典》相邻关系原则）：\n1. **停止侵害**：要求邻居立即停止破坏性施工，修复或加固承重墙。  \n2. **排除妨碍、消除危险**：要求邻居采取加固措施消除安全隐患。  \n3. **恢复原状**：要求将承重墙修复至安全状态。  \n4. **赔偿损失**：包括房屋修复费用、鉴定费、因修缮产生的临时安置费等实际损失。  \n5. **向行政部门举报**：可向住建部门、城管部门投诉，要求对违规装修进行查处。  \n\n---\n\n### 结论：\n提供的上下文**未覆盖相邻关系条款**，需参考《民法典》第七章（第二百八十八条至第二百九十八条）主张权利。实践中，受害人应**及时固定证据**（如现场照片、鉴定报告、沟通记录），并优先通过协商、物业/居委会调解解决；若无效，可向法院提起相邻关系纠纷诉讼。'

In [10]:
ask_llm_with_rag("按份共有人请求分割共有物的法定条件是什么？与共同共有分割规则有何区别？")


====================RAG检索结果====================

[
    [
        "二、权利质权\n\n**第四百四十九条** 可以出质的权利包括：\n（一）汇票、本票、支票；\n（二）债券、存款单；\n（三）仓单、提单；\n（四）可以转让的基金份额、股权；\n（五）可以转让的注册商标专用权、专利权、著作权等知识产权中的财产权；\n（六）应收账款；\n（七）法律、行政法规规定可以出质的其他财产权利。\n\n**第四百五十条** 以汇票、本票、支票、债券、存款单、仓单、提单出质的，当事人应当订立书面合同。质权自权利凭证交付之日起设立。\n\n**第四百五十一条** 以记名股票出质的，当事人应当订立书面合同。质权自股票交付之日起设立。\n以未上市公司股权出质的，适用公司法有关股权转让的规定。\n\n**第四百五十二条** 以可以转让的基金份额、股权出质的，当事人应当订立书面合同。质权自基金份额、股权登记于证券登记结算机构或者公司章程载明的股权登记簿时设立。\n以未上市公司股权出质的，适用公司法有关股权转让的规定。\n\n**第四百五十三条** 以可以转让的注册商标专用权、专利权、著作权等知识产权中的财产权出质的，当事人应当订立书面合同。质权自权利质押登记于相关部门时设立。\n\n**第四百五十四条** 以应收账款出质的，当事人应当订立书面合同。质权自应收账款质押登记于中国人民银行征信中心时设立。\n\n**第四百五十五条** 以法律、行政法规规定可以出质的其他财产权利出质的，依照法律、行政法规的规定。\n\n**第四百五十六条** 权利质权除适用本节规定外，参照适用本章动产质权的有关规定。\n\n####",
        0.5626925826072693
    ],
    [
        "第六节 地役权\n\n**第三百八十八条** 地役权人有权依照合同约定，利用他人的不动产，以提高自己的不动产的效益。\n前款所称他人的不动产为供役地，自己的不动产为需役地。\n\n**第三百八十九条** 地役权合同一般包括下列条款：\n（一）当事人的姓名或者名称和住所；\n（二）供役地和需役地的位置；\n（三）地役权的目的和期限；\n（四）利用供役地的方式；\n（五）报酬及其支付方式；\n（六）争议解决方式。\n\n*

'根据提供的上下文信息，<u>未包含按份共有和共同共有分割规则的具体条文</u>。上下文主要涉及以下内容：\n\n1. **权利质权**（第四百四十九条至第四百五十六条）  \n   规定了可出质权利类型及设立方式（如汇票、股权、知识产权、应收账款等）。\n\n2. **地役权**（第三百八十八条至第三百九十八条）  \n   定义了地役权设立、登记、转让规则及当事人权利义务。\n\n3. **合同变更转让**（第五百四十八条至第五百五十六条）  \n   涵盖债权/债务转让规则、权利义务概括转移及变更后的损害赔偿。\n\n4. **担保物权一般规定**（第三百九十九条至第四百零八条）  \n   明确担保物权范围（抵押权、质权、留置权）、设立登记及消灭条件。\n\n5. **合同权利义务终止**（第五百五十七条至第五百七十条）  \n   列举合同终止情形（履行、解除、抵销等）及终止后义务。\n\n**结论**：  \n上下文未包含 **"按份共有物分割条件"** 或 **"共同共有分割规则"** 的条文，无法根据提供的信息回答该问题。建议补充《民法典》物权编中关于"共有"（第八章）的相关条文（如第三百零三条至第三百一十条），以便进行准确对比分析。'